In [1]:
##This Iteration calculates the QoQ revenue change as the y_tag
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
import scipy.sparse
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

/home/felixtsui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)



access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

# #Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [2]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [73]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
            #if count == 500: break  # initially limit for time sake
    

In [74]:
#Read from the pre prepared vocab file
allword=set()
with open('vocab1.csv', 'r') as f:
    for line in f:
        allword.add(line.strip())

In [75]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  
print("Vocabulary size: {:,}".format(vocab.size))

Vocabulary size: 5,855


In [155]:
#Create the padding
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [476]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [477]:
#Set of all the stocks of interest
#trainstock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
#teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']

trainstock =["TitanMachinery"]
teststock =trainstock


In [478]:
##Converting the string of data into dictionary


for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in trainstock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
                ##Storing in content of each tweet and length of each tweet in sub level variable
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    ##Storing in content of each tweet and length of each tweet in the top level variable
    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [479]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [480]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Calculate change of revenue into the y_label 
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                last_rev=j['value']
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            ##Storing the initial quarter of revenue for calculation
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [481]:
##Writing y_tag to file
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [482]:
##Creating the train/Dev/test var
train_x_init = []
train_n_init = []
train_y_init = []
dev_x_init = []
dev_n_init = []
dev_y_init = []
test_x_init = []
test_n_init = []
test_y_init = []


In [483]:
##Putting the earlier dictinoary into the train var in order of quarter
trainquarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

for i in trainstock:
    for j in range(len(trainquarter)):
        try:
                train_x_init.append(InputX [i][trainquarter[j]])
                train_n_init.append(InputN [i][trainquarter[j]])
                length=len(InputN [i][trainquarter[j]])           
                train_y_init.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass



In [484]:
##Putting the earlier dictinoary into the dev/test var in order of quarter

testquarter=['18Q2','18Q3']

for i in teststock:
    for j in range(len(testquarter)):
        try:
            if testquarter[j]=='18Q3':
                test_x_init.append(InputX [i][testquarter[j]])
                test_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                test_y_init.append([Outputy [i]['revenuechange'][j]]*length)   
            
            elif testquarter[j]=='18Q2':
                dev_x_init.append(InputX [i][testquarter[j]])
                dev_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                dev_y_init.append([Outputy [i]['revenuechange'][j]]*length)     
               
      
        except:
            pass

In [485]:
##Creating the refined train/Dev/test var by flattening the shape of the original var
train_x_init2 = []
train_n_init2 = []
train_y_init2 = []
dev_x_init2 = []
dev_n_init2 = []
dev_y_init2 = []
test_x_init2 = []
test_n_init2 = []
test_y_init2 = []

for i in range(len(train_x_init)):
    for j in range(len(train_x_init[i])):
        train_x_init2.append(train_x_init[i][j])
        train_n_init2.append(train_n_init[i][j])
        train_y_init2.append(train_y_init[i][j])
        
for i in range(len(dev_x_init)):
    for j in range(len(dev_x_init[i])):
        dev_x_init2.append(dev_x_init[i][j])
        dev_n_init2.append(dev_n_init[i][j])
        dev_y_init2.append(dev_y_init[i][j])
        
for i in range(len(test_x_init)):
    for j in range(len(test_x_init[i])):
        test_x_init2.append(test_x_init[i][j])
        test_n_init2.append(test_n_init[i][j])
        test_y_init2.append(test_y_init[i][j])



# NEURAL BAG OF WORDS MODEL #

In [486]:
##Padding the refined var
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr 

In [487]:
##Creating the final refined train/Dev/test var by padding it
train_x = pad_np_array2(train_x_init2, max_len = 50, pad_id=0)
train_n = np.array(train_n_init2)
train_y = np.array(train_y_init2)

test_x = pad_np_array2(test_x_init2, max_len = 50, pad_id=0)
test_n = np.array(test_n_init2)
test_y = np.array(test_y_init2)

dev_x = pad_np_array2(dev_x_init2, max_len = 50, pad_id=0)
dev_n = np.array(dev_n_init2)
dev_y = np.array(dev_y_init2)

In [488]:
##Setting up the NBoW Model
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 125

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 125 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

15,625 examples, moving-average loss 0.40
31,250 examples, moving-average loss 0.30
46,875 examples, moving-average loss 1.07
62,500 examples, moving-average loss 0.31
78,125 examples, moving-average loss 0.32
93,750 examples, moving-average loss 0.30
109,375 examples, moving-average loss 0.29
125,000 examples, moving-average loss 0.29
140,625 examples, moving-average loss 1.15
156,250 examples, moving-average loss 0.31
171,875 examples, moving-average loss 0.35
187,500 examples, moving-average loss 0.31
203,125 examples, moving-average loss 0.30
218,750 examples, moving-average loss 0.36
234,375 examples, moving-average loss 0.32
250,000 examples, moving-average loss 0.31
265,625 examples, moving-average loss 0.30
281,250 examples, moving-average loss 0.64
296,875 examples, moving-average loss 0.33
Completed one epoch in 0:00:12


In [489]:
##Setting up the NBoW Model
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (5,855 words) written to '/tmp/tf_bow_sst_20181209-0002/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20181209-0002/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181209-0002', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f27f2a11c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181209-0002' --port 6006

Then in your browser, open: http:

In [490]:
##Training the NBoW Model
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=125, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tf_bow_sst_20181209-0002/model.ckpt.
INFO:tensorflow:loss = 5.417704, step = 1
INFO:tensorflow:global_step/sec: 141.593
INFO:tensorflow:loss = 0.53488797, step = 101 (0.709 sec)
INFO:tensorflow:global_step/sec: 163.544
INFO:tensorflow:loss = 0.3135513, step = 201 (0.612 sec)
INFO:tensorflow:global_step/sec: 156.025
INFO:tensorflow:loss = 0.29606923, step = 301 (0.642 sec)
INFO:tensorflow:global_step/sec: 160.44
INFO:tensorflow:loss = 0.29183567, step = 401 (0.622 sec)
INFO:tensorflow:global_step/sec: 160.549
INFO:tensorflow:loss = 0.2904357, step = 501 (0.623 sec)
INFO:tensorflow:global_step/sec: 160.896
INFO:tensorflow:loss = 0.28982428, step = 601 (0.621 sec)
INFO:tensorflow:global_step/sec: 158.802
I

INFO:tensorflow:loss = 0.3214141, step = 6773 (0.611 sec)
INFO:tensorflow:global_step/sec: 160.018
INFO:tensorflow:loss = 0.31240624, step = 6873 (0.627 sec)
INFO:tensorflow:global_step/sec: 157.258
INFO:tensorflow:loss = 0.30708855, step = 6973 (0.637 sec)
INFO:tensorflow:global_step/sec: 157.787
INFO:tensorflow:loss = 1.0965147, step = 7073 (0.630 sec)
INFO:tensorflow:global_step/sec: 161.999
INFO:tensorflow:loss = 0.35953578, step = 7173 (0.618 sec)
INFO:tensorflow:global_step/sec: 156.7
INFO:tensorflow:loss = 0.47591847, step = 7273 (0.639 sec)
INFO:tensorflow:global_step/sec: 177.237
INFO:tensorflow:loss = 0.33468172, step = 7373 (0.564 sec)
INFO:tensorflow:global_step/sec: 163.084
INFO:tensorflow:loss = 0.31908253, step = 7473 (0.611 sec)
INFO:tensorflow:global_step/sec: 175.964
INFO:tensorflow:loss = 0.31072655, step = 7573 (0.568 sec)
INFO:tensorflow:global_step/sec: 173.342
INFO:tensorflow:loss = 0.30570784, step = 7673 (0.578 sec)
INFO:tensorflow:global_step/sec: 166.267
INFO

INFO:tensorflow:global_step/sec: 150.37
INFO:tensorflow:loss = 0.3511131, step = 13845 (0.665 sec)
INFO:tensorflow:global_step/sec: 156.043
INFO:tensorflow:loss = 0.33323377, step = 13945 (0.641 sec)
INFO:tensorflow:global_step/sec: 153.777
INFO:tensorflow:loss = 0.32222474, step = 14045 (0.650 sec)
INFO:tensorflow:global_step/sec: 148.749
INFO:tensorflow:loss = 0.31494725, step = 14145 (0.674 sec)
INFO:tensorflow:global_step/sec: 126.276
INFO:tensorflow:loss = 0.81764257, step = 14245 (0.790 sec)
INFO:tensorflow:Saving checkpoints for 14316 into /tmp/tf_bow_sst_20181209-0002/model.ckpt.
INFO:tensorflow:Loss for final step: 0.38111705.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-09-00:03:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0002/model.ckpt-14316
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow

INFO:tensorflow:loss = 0.31647646, step = 19589 (0.693 sec)
INFO:tensorflow:global_step/sec: 148.884
INFO:tensorflow:loss = 0.3115633, step = 19689 (0.672 sec)
INFO:tensorflow:global_step/sec: 153.666
INFO:tensorflow:loss = 0.47078407, step = 19789 (0.651 sec)
INFO:tensorflow:global_step/sec: 148.47
INFO:tensorflow:loss = 0.374605, step = 19889 (0.672 sec)
INFO:tensorflow:global_step/sec: 150.886
INFO:tensorflow:loss = 0.3669871, step = 19989 (0.663 sec)
INFO:tensorflow:global_step/sec: 152.597
INFO:tensorflow:loss = 0.3360263, step = 20089 (0.656 sec)
INFO:tensorflow:global_step/sec: 149.146
INFO:tensorflow:loss = 0.3246935, step = 20189 (0.671 sec)
INFO:tensorflow:global_step/sec: 147.606
INFO:tensorflow:loss = 1.0380499, step = 20289 (0.677 sec)
INFO:tensorflow:global_step/sec: 153.57
INFO:tensorflow:loss = 0.3884426, step = 20389 (0.651 sec)
INFO:tensorflow:global_step/sec: 156.359
INFO:tensorflow:loss = 0.35737428, step = 20489 (0.640 sec)
INFO:tensorflow:global_step/sec: 154.622


In [491]:
##Calculating the Accuracy

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  

print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-09-00:04:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0002/model.ckpt-23860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-09-00:04:50
INFO:tensorflow:Saving dict for global step 23860: accuracy = 0.0, cross_entropy_loss = 3.1433702, global_step = 23860, loss = 3.4453971
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 23860: /tmp/tf_bow_sst_20181209-0002/model.ckpt-23860
Accuracy on test set: 0.00%


{'accuracy': 0.0,
 'cross_entropy_loss': 3.1433702,
 'loss': 3.4453971,
 'global_step': 23860}

In [492]:
##Calculating the Precision
from sklearn.metrics import average_precision_score
    
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
average_precision = average_precision_score(y_pred,test_y)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181209-0002/model.ckpt-23860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Average precision score: 1.00


In [493]:
##Loss Analysis
counttest=[]

for i in range(len(test_y_init)):
    counttest.append(len(test_y_init[i]))
    
count=0
for i in range(len(counttest)):
 
    lbl, cnts = np.unique(test_y[count:count+counttest[i]], return_counts=True)
    lbl2, cnts2 = np.unique (y_pred[count:count+counttest[i]], return_counts=True)
    print(teststock[i])
    print(list(zip(lbl2, cnts2)))
    print(lbl[0], "test vs predict", round(sum(lbl2*cnts2)/sum(cnts2),0))
    count+=counttest[i]



TitanMachinery
[(1, 22934)]
0 vs 1.0
